# THIS NOTEBOOK IS NOT QUITE FINISHED YET

The aim is to illustrate the usage of `naklar`

In [18]:
# modified version of
# http://scikit-learn.org/stable/auto_examples/svm/plot_iris.html#example-svm-plot-iris-py

import os
import pickle
import itertools

import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets

from naklar import experiment


# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features. We could
                      # avoid this ugly slicing by using a two-dim dataset
y = iris.target

def evaluate_settings(i, output='results', **kwargs):
    # run experiment
    svc = svm.SVC(**kwargs).fit(X, y)
    pred = svc.predict(X)
    
    # store results on disk
    conf = kwargs
    conf['results'] = pred # results is a NumPy array
    if not os.path.exists(os.path.join('.', output, str(i))):
        os.makedirs(os.path.join('.', output, str(i)))
    conf['path'] = os.path.join(output, str(i))
    with open(os.path.join(output, str(i), 'conf.pkl'), 'wb') as out:
        pickle.dump(conf, out)
    
    # store the trained model as well
    with open(os.path.join(output, str(i), 'svc.skl'), 'wb') as out:
        pickle.dump(svc, out)


In [43]:
kernels = ['linear', 'rbf', 'poly']
Cs = [0.001, 0.1, 1, 2]

# could also use sklearn.cross_validation
jobs = itertools.product(kernels, Cs)
for i, (kernel, C) in enumerate(jobs):
    evaluate_settings(i, kernel=kernel, C=C)

experiment.initialise('results/', autoload=True)

print(experiment.select())

experiment.select(C=1)
for exp in experiment.select(C=1):
    print(exp.C, exp.kernel, exp.path)